# 特殊なデータ型の操作

In [1]:
from datetime import date
import os

import polars as pl

## 定数定義

In [2]:
DATA_PAR_PATH = os.path.join('..','..','data')
INPUT_CSV_PATH_STOCKS = os.path.join(DATA_PAR_PATH,'all_stocks.csv')

## 文字列

In [3]:
df = pl.DataFrame({
    'raw_text': [
        '  Data Science is amazing',
        'Data_analysis > Data entry',
        ' Python&Polars; False'
    ]
})

print(df)

shape: (3, 1)
┌────────────────────────────┐
│ raw_text                   │
│ ---                        │
│ str                        │
╞════════════════════════════╡
│   Data Science is amazing  │
│ Data_analysis > Data entry │
│  Python&Polars; False      │
└────────────────────────────┘


In [4]:
df = df.with_columns(
    pl.col('raw_text')
    .str.strip_chars()
    .str.to_lowercase()
    .str.replace_all('_', ' ')
    .alias('processed_text')
)

print(df)

shape: (3, 2)
┌────────────────────────────┬────────────────────────────┐
│ raw_text                   ┆ processed_text             │
│ ---                        ┆ ---                        │
│ str                        ┆ str                        │
╞════════════════════════════╪════════════════════════════╡
│   Data Science is amazing  ┆ data science is amazing    │
│ Data_analysis > Data entry ┆ data analysis > data entry │
│  Python&Polars; False      ┆ python&polars; false       │
└────────────────────────────┴────────────────────────────┘


In [5]:
print(
    df.with_columns(
        pl.col('processed_text')
        .str.slice(0, 5)
        .alias('first_5_chars'),
        pl.col('processed_text')
        .str.split(' ')
        .list.get(0)
        .alias('first_word'),
        pl.col('processed_text')
        .str.split(' ')
        .list.get(1)
        .alias('second_word')
    )
)

shape: (3, 5)
┌─────────────────────────┬─────────────────────────┬───────────────┬────────────────┬─────────────┐
│ raw_text                ┆ processed_text          ┆ first_5_chars ┆ first_word     ┆ second_word │
│ ---                     ┆ ---                     ┆ ---           ┆ ---            ┆ ---         │
│ str                     ┆ str                     ┆ str           ┆ str            ┆ str         │
╞═════════════════════════╪═════════════════════════╪═══════════════╪════════════════╪═════════════╡
│ Data Science is amazing ┆ data science is amazing ┆ data          ┆ data           ┆ science     │
│ Data_analysis > Data    ┆ data analysis > data    ┆ data          ┆ data           ┆ analysis    │
│ entry                   ┆ entry                   ┆               ┆                ┆             │
│  Python&Polars; False   ┆ python&polars; false    ┆ pytho         ┆ python&polars; ┆ false       │
└─────────────────────────┴─────────────────────────┴───────────────┴────────

In [6]:
print(
    df.with_columns(
        pl.col('processed_text')
        .str.len_chars()
        .alias('amount_of_chars'),
        pl.col('processed_text')
        .str.len_bytes()
        .alias('amout_of_bytes'),
        pl.col('processed_text')
        .str.count_matches('a')
        .alias('count_a')
    )
)

shape: (3, 5)
┌──────────────────────────┬──────────────────────────┬─────────────────┬────────────────┬─────────┐
│ raw_text                 ┆ processed_text           ┆ amount_of_chars ┆ amout_of_bytes ┆ count_a │
│ ---                      ┆ ---                      ┆ ---             ┆ ---            ┆ ---     │
│ str                      ┆ str                      ┆ u32             ┆ u32            ┆ u32     │
╞══════════════════════════╪══════════════════════════╪═════════════════╪════════════════╪═════════╡
│ Data Science is amazing  ┆ data science is amazing  ┆ 23              ┆ 23             ┆ 4       │
│ Data_analysis > Data     ┆ data analysis > data     ┆ 26              ┆ 26             ┆ 6       │
│ entry                    ┆ entry                    ┆                 ┆                ┆         │
│  Python&Polars; False    ┆ python&polars; false     ┆ 20              ┆ 20             ┆ 2       │
└──────────────────────────┴──────────────────────────┴─────────────────┴────

`.len_chars()`の計算量は$\mathcal{O}(n)$であるが、`.len_bytes()`は$\mathcal{O}(1)$とのこと。  
使い方次第では処理時間に大きく起因するので、これを知れたのは良い機会

## カテゴリカル

In [7]:
df1 = pl.DataFrame(
    {'categorical_column': ['value1', 'value2', 'value3']},
    schema={'categorical_column': pl.Categorical},
)

print(
    df1.with_columns(
        pl.col('categorical_column')
        .to_physical()
        .alias('categorical_column_physical')
    )
)

shape: (3, 2)
┌────────────────────┬─────────────────────────────┐
│ categorical_column ┆ categorical_column_physical │
│ ---                ┆ ---                         │
│ cat                ┆ u32                         │
╞════════════════════╪═════════════════════════════╡
│ value1             ┆ 0                           │
│ value2             ┆ 1                           │
│ value3             ┆ 2                           │
└────────────────────┴─────────────────────────────┘


上のセルの出力は、string型のカテゴリカルな値を、物理表現（=int型での表現）にエンコードしているという操作

In [8]:
df2 = pl.DataFrame(
    {'categorical_column': ['value4', 'value3', 'value2']},
    schema={'categorical_column': pl.Categorical}
)

print(
    df2.with_columns(
        pl.col('categorical_column')
        .to_physical()
        .alias('categorical_column_physical')
    )
)

shape: (3, 2)
┌────────────────────┬─────────────────────────────┐
│ categorical_column ┆ categorical_column_physical │
│ ---                ┆ ---                         │
│ cat                ┆ u32                         │
╞════════════════════╪═════════════════════════════╡
│ value4             ┆ 0                           │
│ value3             ┆ 1                           │
│ value2             ┆ 2                           │
└────────────────────┴─────────────────────────────┘


In [9]:
df1.join(df2, on='categorical_column')

/var/folders/hm/1554tfbj14b8r1wpst4sml2h0000gn/T/ipykernel_95880/2194104470.py:1: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance
  df1.join(df2, on='categorical_column')


categorical_column
cat
"""value3"""
"""value2"""


上のwarningは、結合しようとしているカテゴリ値が別々に物理表現にエンコードされていて、結合時に両方とも再エンコードしてしまい、計算コストがかかってしまうよ。的なエラー。  
この解消方法として、`グローバル文字列キャッシュ`という仕組みを利用する。  
これは、すべてのカテゴリ間で共有される文字列キャッシュのことで、このカテゴリ値をグローバル化することで、再エンコードしなくてよくなる。  
ただし、すべてのカテゴリ値がグローバル文字列キャッシュを使用すると、かえってパフォーマスが低下する（大量にデータがあるとそうなるのは当然か）。そのため、デフォルトではオフになっている仕組み

In [10]:
with pl.StringCache():
    df1 = pl.DataFrame(
        {
            'categorical_column': ['value3', 'value2', 'value1'],
            'other': ['a', 'b', 'c']
        },
        schema={'categorical_column': pl.Categorical, 'other': pl.String}
    )
    df2 = pl.DataFrame(
        {
            'categorical_column': ['value2', 'value3', 'value4'],
            'other': ['d', 'e', 'f']
        },
        schema={'categorical_column': pl.Categorical, 'other': pl.String}
    )

df1.join(df2, on='categorical_column')

categorical_column,other,other_right
cat,str,str
"""value2""","""b""","""d"""
"""value3""","""a""","""e"""


In [11]:
# pl.enable_string_cache()

上のセルを実行すると、スクリプト全体に対して常に`グローバル文字列キャッシュ`が有効になる。  
使い所の見極めは大事だな

In [12]:
df2.select(pl.col('categorical_column').cat.get_categories())

categorical_column
str
"""value2"""
"""value3"""
"""value4"""


In [13]:
sorting_comparison_df = (
    df2
    .select(
        pl.col('categorical_column')
        .alias('categorical_lexical')
    )
    .with_columns(
        pl.col('categorical_lexical')
        .to_physical()
        .alias('categorical_physical')
    )
)

print(sorting_comparison_df)

shape: (3, 2)
┌─────────────────────┬──────────────────────┐
│ categorical_lexical ┆ categorical_physical │
│ ---                 ┆ ---                  │
│ cat                 ┆ u32                  │
╞═════════════════════╪══════════════════════╡
│ value2              ┆ 1                    │
│ value3              ┆ 0                    │
│ value4              ┆ 3                    │
└─────────────────────┴──────────────────────┘


In [14]:
print(
    sorting_comparison_df
    .with_columns(
        pl.col('categorical_lexical')
        .cast(pl.Categorical('physical'))
    )
    .sort(by='categorical_lexical')
)

shape: (3, 2)
┌─────────────────────┬──────────────────────┐
│ categorical_lexical ┆ categorical_physical │
│ ---                 ┆ ---                  │
│ cat                 ┆ u32                  │
╞═════════════════════╪══════════════════════╡
│ value3              ┆ 0                    │
│ value2              ┆ 1                    │
│ value4              ┆ 3                    │
└─────────────────────┴──────────────────────┘


In [15]:
print(
    sorting_comparison_df
    .with_columns(
        pl.col('categorical_lexical')
        .cast(pl.Categorical('lexical'))
    )
    .sort(by='categorical_lexical')
)

shape: (3, 2)
┌─────────────────────┬──────────────────────┐
│ categorical_lexical ┆ categorical_physical │
│ ---                 ┆ ---                  │
│ cat                 ┆ u32                  │
╞═════════════════════╪══════════════════════╡
│ value2              ┆ 1                    │
│ value3              ┆ 0                    │
│ value4              ┆ 3                    │
└─────────────────────┴──────────────────────┘


In [16]:
enum_dtype = pl.Enum(['Polar', 'Panda', 'Brown'])
enum_series = pl.Series(
    ['Polar', 'Panda', 'Brown', 'Brown', 'Polar'], dtype=enum_dtype
)

cat_series = pl.Series(
    ['Polar', 'Panda', 'Brown', 'Brown', 'Polar'], dtype=pl.Categorical
)

列挙型はpolarsで使える新しい型

## 時間データ

In [17]:
pl.read_csv(INPUT_CSV_PATH_STOCKS, try_parse_dates=True)

symbol,date,open,high,low,close,adj close,volume
str,date,f64,f64,f64,f64,f64,i64
"""NVDA""",2020-01-02,59.6875,59.977501,59.18,59.977501,59.744038,23753600
"""NVDA""",2020-01-03,58.775002,59.4575,58.525002,59.017502,58.787781,20538400
"""NVDA""",2020-01-06,58.080002,59.317501,57.817501,59.264999,59.034313,26263600
"""NVDA""",2020-01-07,59.549999,60.442501,59.0975,59.982498,59.749023,31485600
"""NVDA""",2020-01-08,59.939999,60.509998,59.537498,60.095001,59.861084,27710800
…,…,…,…,…,…,…,…
"""NVDA""",2023-06-26,424.609985,427.640015,401.0,406.320007,406.250824,59432200
"""NVDA""",2023-06-27,407.98999,419.399994,404.480011,418.76001,418.688721,46217500
"""NVDA""",2023-06-28,406.600006,418.450012,405.179993,411.170013,411.100006,58263900


`try_parse_dates=True`にすることで、読み込み時に時間データであれば、データ型をstringではなく、date型やdatetime型で読み込んでくれる

In [18]:
df = pl.DataFrame({
    'date_str': ['2023-12-31', '2024-02-29']
})

df = df.with_columns(
    pl.col('date_str').str.strptime(pl.Date, '%Y-%m-%d').alias('date')
)

print(df)

shape: (2, 2)
┌────────────┬────────────┐
│ date_str   ┆ date       │
│ ---        ┆ ---        │
│ str        ┆ date       │
╞════════════╪════════════╡
│ 2023-12-31 ┆ 2023-12-31 │
│ 2024-02-29 ┆ 2024-02-29 │
└────────────┴────────────┘


In [19]:
df = df.with_columns(
    pl.col('date').dt.to_string('%d-%m-%Y').alias('formatted_date')
)

print(df)

shape: (2, 3)
┌────────────┬────────────┬────────────────┐
│ date_str   ┆ date       ┆ formatted_date │
│ ---        ┆ ---        ┆ ---            │
│ str        ┆ date       ┆ str            │
╞════════════╪════════════╪════════════════╡
│ 2023-12-31 ┆ 2023-12-31 ┆ 31-12-2023     │
│ 2024-02-29 ┆ 2024-02-29 ┆ 29-02-2024     │
└────────────┴────────────┴────────────────┘


年月日の並び替えもできるのか

In [20]:
df = pl.DataFrame(
    {
        'date': pl.date_range(
            start=date(2023, 12, 31),
            end=date(2024, 1, 15),
            interval='1w',
            eager=True
        )
    }
)

print(df)

shape: (3, 1)
┌────────────┐
│ date       │
│ ---        │
│ date       │
╞════════════╡
│ 2023-12-31 │
│ 2024-01-07 │
│ 2024-01-14 │
└────────────┘


`.date_range()`って使い所次第ではすごく助かるなぁ

In [21]:
df = pl.DataFrame(
    {
        'utc_mixed_offset_data': [
            '2021-03-27T00:00:00+0100',
            '2021-03-28T00:00:00+0100',
            '2021-03-29T00:00:00+0100',
            '2021-03-30T00:00:00+0100'
        ]
    }
)
df = (
    df.with_columns(
        pl.col('utc_mixed_offset_data')
        .str.to_datetime('%Y-%m-%dT%H:%M:%S%z')
        .alias('parsed_data')
    ).with_columns(
        pl.col('parsed_data')
        .dt.convert_time_zone('Europe/Amsterdam')
        .alias('converted_data')
    )
)

print(df)

shape: (4, 3)
┌──────────────────────────┬─────────────────────────┬────────────────────────────────┐
│ utc_mixed_offset_data    ┆ parsed_data             ┆ converted_data                 │
│ ---                      ┆ ---                     ┆ ---                            │
│ str                      ┆ datetime[μs, UTC]       ┆ datetime[μs, Europe/Amsterdam] │
╞══════════════════════════╪═════════════════════════╪════════════════════════════════╡
│ 2021-03-27T00:00:00+0100 ┆ 2021-03-26 23:00:00 UTC ┆ 2021-03-27 00:00:00 CET        │
│ 2021-03-28T00:00:00+0100 ┆ 2021-03-27 23:00:00 UTC ┆ 2021-03-28 00:00:00 CET        │
│ 2021-03-29T00:00:00+0100 ┆ 2021-03-28 23:00:00 UTC ┆ 2021-03-29 01:00:00 CEST       │
│ 2021-03-30T00:00:00+0100 ┆ 2021-03-29 23:00:00 UTC ┆ 2021-03-30 01:00:00 CEST       │
└──────────────────────────┴─────────────────────────┴────────────────────────────────┘


タイムゾーンの切り替えは場所を指定するだけか。時差を指定してコメントで所在地教えるでも良いけれど、こっちの方がやや綺麗かも？

## リスト

In [22]:
bool_df = pl.DataFrame({
    'values': [[True, True], [False, False, True], [False]]
})

print(
    bool_df
    .with_columns(
        pl.col('values')
        .list.all()
        .alias('all values'),
        pl.col('values')
        .list.any()
        .alias('any values')
    )
)

shape: (3, 3)
┌──────────────────────┬────────────┬────────────┐
│ values               ┆ all values ┆ any values │
│ ---                  ┆ ---        ┆ ---        │
│ list[bool]           ┆ bool       ┆ bool       │
╞══════════════════════╪════════════╪════════════╡
│ [true, true]         ┆ true       ┆ true       │
│ [false, false, true] ┆ false      ┆ true       │
│ [false]              ┆ false      ┆ false      │
└──────────────────────┴────────────┴────────────┘


In [23]:
df = pl.DataFrame({
    'values': [[10, 20], [30, 40, 50], [60]]
})

print(
    df
    .with_columns(
        pl.col('values')
        .list.eval(
            pl.element() > 40,
            parallel=True
        )
        .alias('values > 40')
    )
    .with_columns(
        pl.col('values > 40')
        .list.all()
        .alias('all values > 40')
    )
)

shape: (3, 3)
┌──────────────┬──────────────────────┬─────────────────┐
│ values       ┆ values > 40          ┆ all values > 40 │
│ ---          ┆ ---                  ┆ ---             │
│ list[i64]    ┆ list[bool]           ┆ bool            │
╞══════════════╪══════════════════════╪═════════════════╡
│ [10, 20]     ┆ [false, false]       ┆ false           │
│ [30, 40, 50] ┆ [false, false, true] ┆ false           │
│ [60]         ┆ [true]               ┆ true            │
└──────────────┴──────────────────────┴─────────────────┘


`parallel=True`によって、並列処理ができる。手軽でいいね

In [24]:
df.explode('values')

values
i64
10
20
30
40
50
60


## 配列
リストと紛らわしいが、ここで言う配列はArray型のこと

In [25]:
df = pl.DataFrame([
    pl.Series(
        'location',
        ['Paris', 'Amsterdam', 'Barcelona'],
        dtype=pl.String
    ),
    pl.Series(
        'temperatures',
        [
            [23, 27, 21, 22, 24, 23, 22],
            [17, 19, 15, 22, 18, 20, 21],
            [30, 32, 28, 29, 34, 33, 31]
        ],
        dtype=pl.Array(pl.Int64, shape=7)
    )
])

print(df)

shape: (3, 2)
┌───────────┬────────────────┐
│ location  ┆ temperatures   │
│ ---       ┆ ---            │
│ str       ┆ array[i64, 7]  │
╞═══════════╪════════════════╡
│ Paris     ┆ [23, 27, … 22] │
│ Amsterdam ┆ [17, 19, … 21] │
│ Barcelona ┆ [30, 32, … 31] │
└───────────┴────────────────┘


In [26]:
print(
    df
    .with_columns(
        pl.col('temperatures')
        .arr.median()
        .alias('median'),
        pl.col('temperatures')
        .arr.max()
        .alias('max'),
        pl.col('temperatures')
        .arr.arg_max()
        .alias('warmest_weekday')
    )
)

shape: (3, 5)
┌───────────┬────────────────┬────────┬─────┬─────────────────┐
│ location  ┆ temperatures   ┆ median ┆ max ┆ warmest_weekday │
│ ---       ┆ ---            ┆ ---    ┆ --- ┆ ---             │
│ str       ┆ array[i64, 7]  ┆ f64    ┆ i64 ┆ u32             │
╞═══════════╪════════════════╪════════╪═════╪═════════════════╡
│ Paris     ┆ [23, 27, … 22] ┆ 23.0   ┆ 27  ┆ 1               │
│ Amsterdam ┆ [17, 19, … 21] ┆ 19.0   ┆ 22  ┆ 3               │
│ Barcelona ┆ [30, 32, … 31] ┆ 31.0   ┆ 34  ┆ 4               │
└───────────┴────────────────┴────────┴─────┴─────────────────┘


## 構造体
struct型のこと

In [27]:
df = pl.DataFrame({
    'struct_column': [
        {'a': 1, 'b': 2},
        {'a': 3, 'b': 4},
        {'a': 5, 'b': 6}
    ]
})

print(df)

shape: (3, 1)
┌───────────────┐
│ struct_column │
│ ---           │
│ struct[2]     │
╞═══════════════╡
│ {1,2}         │
│ {3,4}         │
│ {5,6}         │
└───────────────┘


以下、おさらい
* list型  -> pythonで通常使うプリミティブ型のリスト
* array型 -> pythonだと`import Array`しないといけないリスト
    * list型は要素の型がバラバラで良いけれど、array型は要素の方が統一されている必要がある
* struct  -> DataFrameの中に入れる辞書型のこと

In [28]:
df.select(pl.col('struct_column').struct.field('a'))

a
i64
1
3
5


In [29]:
df = df.unnest('struct_column')
print(df)

shape: (3, 2)
┌─────┬─────┐
│ a   ┆ b   │
│ --- ┆ --- │
│ i64 ┆ i64 │
╞═════╪═════╡
│ 1   ┆ 2   │
│ 3   ┆ 4   │
│ 5   ┆ 6   │
└─────┴─────┘


In [30]:
df.select(
    'a',
    'b',
    pl.struct(
        pl.col('a'),
        pl.col('b')
    ).alias('struct_column')
)

a,b,struct_column
i64,i64,struct[2]
1,2,"{1,2}"
3,4,"{3,4}"
5,6,"{5,6}"


In [31]:
df = pl.DataFrame({
    'fruit': ['cherry', 'apple', 'banana', 'banana', 'apple', 'banana']
})

print(df)

shape: (6, 1)
┌────────┐
│ fruit  │
│ ---    │
│ str    │
╞════════╡
│ cherry │
│ apple  │
│ banana │
│ banana │
│ apple  │
│ banana │
└────────┘


In [32]:
print(
    df
    .select(
        pl.col('fruit')
        .value_counts(sort=True)
    )
)

shape: (3, 1)
┌──────────────┐
│ fruit        │
│ ---          │
│ struct[2]    │
╞══════════════╡
│ {"banana",3} │
│ {"apple",2}  │
│ {"cherry",1} │
└──────────────┘


In [33]:
print(
    df.select(
        pl.col('fruit')
        .value_counts(sort=True)
    )
    .unnest('fruit')
)

shape: (3, 2)
┌────────┬───────┐
│ fruit  ┆ count │
│ ---    ┆ ---   │
│ str    ┆ u32   │
╞════════╪═══════╡
│ banana ┆ 3     │
│ apple  ┆ 2     │
│ cherry ┆ 1     │
└────────┴───────┘


`.value_counts()`で算出されたvalueに対して`.unnest()`を実行すると、カラム名は自動的に`count`になるのか